In [1]:
import os
import time


import numpy as np
import scipy as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from PIL import Image, ImageFile
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from layers import ContentLoss, StyleLoss, Normalization
from utils import gram_matrix, loader, unloader, image_loader, imshow
from utils import plot_loss_history, create_collage
from create_train_model import create_nst_model, get_input_optimizer, run_style_transfer

ImageFile.MAXBLOCK = 2**20
%matplotlib inline

#### defualt settings

In [2]:
history_loss = []

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
content_layers_default = ["conv4_2"]
style_layers_default = ["conv1_1", "conv2_1", "conv3_1", "conv4_1", "conv5_1"]

In [5]:
vgg_19_layers_names = {
    0: "conv1_1",
    1: "relu1_1",
    2: "conv1_2",
    3: "relu1_2",
    4: "pool1",
    5: "conv2_1",
    6: "relu2_1",
    7: "conv2_2",
    8: "relu2_2",
    9: "pool2",
    10: "conv3_1",
    11: "relu3_1",
    12: "conv3_2",
    13: "relu3_2",
    14: "conv3_3",
    15: "relu3_3",
    16: "conv3_4",
    17: "relu3_4",
    18: "pool3",
    19: "conv4_1",
    20: "relu4_1",
    21: "conv4_2",
    22: "relu4_2",
    23: "conv4_3",
    24: "relu4_3",
    25: "conv4_4",
    26: "relu4_4",
    27: "pool4",
    28: "conv5_1",
    29: "relu5_1",
    30: "conv5_2",
    31: "relu5_2",
    32: "conv5_3",
    33: "relu5_3",
    34: "conv5_4",
    35: "relu5_4",
    36: "pool5"
}

In [ ]:
vgg19_pretrained = torch.hub.load('pytorch/vision:v0.9.0', 'vgg19', pretrained=True).to(device)

In [7]:
vgg19_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
vgg19_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

In [8]:
imgsize = 256

#### neural style transfer

In [9]:
path_to_content = # path to content image
path_to_style = # path to style image

In [10]:
content_img = image_loader(path_to_content)
style_img = image_loader(path_to_style)

In [ ]:
# plot style and content images
input_img = content_img.clone() # input image is content image, because cold start(like Gaussian noise) is worse than this method

plt.figure()
imshow(input_img, title='input image')

plt.figure()
imshow(style_img, title='style image')

In [ ]:
output = run_style_transfer(
    pretrained_vgg19_model=vgg19_pretrained,
    content_img=content_img,
    style_img=style_img,
    save_folder=None, # folder where you want to save results of nst steps
    normalization_mean=vgg19_normalization_mean,
    normalization_std=vgg19_normalization_std,
    input_img=input_img,
    num_steps=2000,
    style_weight=1e7, # variants: 1e4, 1e5, 1e6, 1e7
    content_weight=1,
    return_history_loss=False)